# Practice Session 08: Data streams

Author: <font color="blue">José M. Pérez Clar</font>

E-mail: <font color="blue">josemaria.perez02@estudiant.upf.edu</font>

Date: <font color="blue">22/11/2023</font>

In [1]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip

# 0. Dataset and how to iterate

In [2]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

In [3]:
# Leave this code as-is

# Producer in Python that reads a filename by words
def read_by_words(filename, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Regular expression to identify words having 3 letters or more and beginning with a-z
        word_expr = re.compile('^[a-z]{2,}$', re.IGNORECASE)

        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for word in nltk.word_tokenize(text):
                          
                if word_expr.match(word):
                    counter += 1
                    
                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [4]:
# Leave this code as-is

# Iterate through the file
for word in read_by_words(INPUT_FILE, max_words=300000, report_every=100000):
    # Prints 1/10000 of words
    if random.random() < 0.0001:
        print("Current word '%s'" % (word)) 

Current word 'work'
Current word 'you'
Current word 'slung'
Current word 'awake'
Current word 'stop'
- Read 100000/300000 words so far
Current word 'to'
Current word 'cover'
Current word 'about'
Current word 'think'
Current word 'had'
Current word 'grandparents'
Current word 'to'
Current word 'help'
Current word 'purified'
Current word 'there'
- Read 200000/300000 words so far
Current word 'see'
Current word 'says'
Current word 'have'
Current word 'it'
Current word 'woman'
Current word 'here'
Current word 'huntin'
Current word 'take'
Current word 'well'
Current word 'with'
Current word 'the'
Current word 'it'
Current word 'is'
- Read 300000/300000 words so far


In [ ]:
# Run this if above gives an error about 'punkt'
#nltk.download('punkt')

# 1. Determine approximately the top-10 words

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

In [7]:
def add_reservoir(reservoir, item, max_reservoir_size):
 
    s = len(reservoir)

    if s < max_reservoir_size:
        reservoir.append(item)
        
    else: #choose randomly the element to replace
        replace_index = random.randint(0, max_reservoir_size - 1)
        reservoir.pop(replace_index)
        reservoir.append(item)

    assert(len(reservoir) <= max_reservoir_size)

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

In [8]:
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []

    words_read = 0

    for word in read_by_words(filename, max_words=max_words, report_every=report_every):
        
        words_read += 1
        probability = reservoir_size / words_read 

        if random.random() < probability: #decide wether to ignore or not
            add_reservoir(reservoir, word, reservoir_size)
        
    return (words_read, reservoir)

In [9]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=1000000, report_every=100000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far
Number of items seen    : 1000023
Number of items sampled : 1500


In [25]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

71 you
53 the
37 to
32 it
29 what
25 of
24 that
20 me
17 we
17 do


<font size="+1" color="red">Replace this cell with your code to print the top items and their relative frequencies</font>

In [33]:
sorted_items = sorted(freq.items(), key=lambda x: x[1], reverse=True)

total_items = len(reservoir)
relative_frequencies = [(item, frequency / total_items * 100) for item, frequency in sorted_items[:15]]

for item, relative_frequency in relative_frequencies:
    absolute_frequency = freq[item]
    print("The word (", item, ") has absolute frequency =", absolute_frequency, "and relative frequency =", relative_frequency, "%")


The word ( you ) has absolute frequency = 6 and relative frequency = 0.4 %
The word ( the ) has absolute frequency = 5 and relative frequency = 0.33333333333333337 %
The word ( it ) has absolute frequency = 5 and relative frequency = 0.33333333333333337 %
The word ( is ) has absolute frequency = 3 and relative frequency = 0.2 %
The word ( not ) has absolute frequency = 3 and relative frequency = 0.2 %
The word ( just ) has absolute frequency = 2 and relative frequency = 0.13333333333333333 %
The word ( think ) has absolute frequency = 2 and relative frequency = 0.13333333333333333 %
The word ( to ) has absolute frequency = 2 and relative frequency = 0.13333333333333333 %
The word ( ai ) has absolute frequency = 2 and relative frequency = 0.13333333333333333 %
The word ( say ) has absolute frequency = 2 and relative frequency = 0.13333333333333333 %
The word ( do ) has absolute frequency = 2 and relative frequency = 0.13333333333333333 %
The word ( me ) has absolute frequency = 2 and re

<font size="+1" color="red">Increase the max limit of words so that one pass takes no more than 5 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.</font>

In [34]:
reservoir_sizes = [50 , 100, 500, 1000, 1500]

for reservoir_size in reservoir_sizes:
    print("\n\nReservoir Size:", reservoir_size)
    words_read, result_reservoir = reservoir_sampling(INPUT_FILE, reservoir_size, 1500000, 100000) #max words increased
     
    freq = {}
    for item in result_reservoir:
        freq[item] = result_reservoir.count(item)
    
    sorted_items = sorted(freq.items(), key=lambda x: x[1], reverse=True)

    total_items = reservoir_size
    relative_frequencies = [(item, frequency / total_items * 100) for item, frequency in sorted_items[:5]]

    for item, relative_frequency in relative_frequencies:
        absolute_frequency = freq[item]
        print("The word (", item, ") has absolute frequency =", absolute_frequency, "and relative frequency =", relative_frequency, "%")

    

Reservoir Size: 50
- Read 100000/1500000 words so far
- Read 200000/1500000 words so far
- Read 300000/1500000 words so far
- Read 400000/1500000 words so far
- Read 500000/1500000 words so far
- Read 600000/1500000 words so far
- Read 700000/1500000 words so far
- Read 800000/1500000 words so far
- Read 900000/1500000 words so far
- Read 1000000/1500000 words so far
- Read 1100000/1500000 words so far
- Read 1200000/1500000 words so far
- Read 1300000/1500000 words so far
- Read 1400000/1500000 words so far
- Read 1500000/1500000 words so far
The word ( too ) has absolute frequency = 2 and relative frequency = 4.0 %
The word ( the ) has absolute frequency = 2 and relative frequency = 4.0 %
The word ( if ) has absolute frequency = 2 and relative frequency = 4.0 %
The word ( of ) has absolute frequency = 2 and relative frequency = 4.0 %
The word ( in ) has absolute frequency = 2 and relative frequency = 4.0 %
Reservoir Size: 100
- Read 100000/1500000 words so far
- Read 200000/1500000 w

The results become somehwat stable (the top-3 words remain mostly the same), when the reservoir size is at least around 100 words. Before, since the reservoir is too small, the fluctuations of these 3 top words are much much higher. This is understandable since the probability of replacing words for others that might be more common is smaller for smaller reservoirs.

<font size="+1" color="red">Remove the max limit of words and re-run. Replace this cell with a brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

In [35]:
reservoir_sizes = [50 , 100, 500, 1000, 1500]

for reservoir_size in reservoir_sizes:
    print("\n\nReservoir Size:", reservoir_size)
    words_read, result_reservoir = reservoir_sampling(INPUT_FILE, reservoir_size, 1500000000000000000000000, 100000)
     
    freq = {}
    for item in result_reservoir:
        freq[item] = result_reservoir.count(item)
    
    sorted_items = sorted(freq.items(), key=lambda x: x[1], reverse=True)

    total_items = reservoir_size
    relative_frequencies = [(item, frequency / total_items * 100) for item, frequency in sorted_items[:5]]

    for item, relative_frequency in relative_frequencies:
        absolute_frequency = freq[item]
        print("The word (", item, ") has absolute frequency =", absolute_frequency, "and relative frequency =", relative_frequency, "%")

    

Reservoir Size: 50
- Read 100000/1500000000000000000000000 words so far
- Read 200000/1500000000000000000000000 words so far
- Read 300000/1500000000000000000000000 words so far
- Read 400000/1500000000000000000000000 words so far
- Read 500000/1500000000000000000000000 words so far
- Read 600000/1500000000000000000000000 words so far
- Read 700000/1500000000000000000000000 words so far
- Read 800000/1500000000000000000000000 words so far
- Read 900000/1500000000000000000000000 words so far
- Read 1000000/1500000000000000000000000 words so far
- Read 1100000/1500000000000000000000000 words so far
- Read 1200000/1500000000000000000000000 words so far
- Read 1300000/1500000000000000000000000 words so far
- Read 1400000/1500000000000000000000000 words so far
- Read 1500000/1500000000000000000000000 words so far
- Read 1600000/1500000000000000000000000 words so far
- Read 1700000/1500000000000000000000000 words so far
- Read 1800000/1500000000000000000000000 words so far
- Read 1900000/150

- Read 700000/1500000000000000000000000 words so far
- Read 800000/1500000000000000000000000 words so far
- Read 900000/1500000000000000000000000 words so far
- Read 1000000/1500000000000000000000000 words so far
- Read 1100000/1500000000000000000000000 words so far
- Read 1200000/1500000000000000000000000 words so far
- Read 1300000/1500000000000000000000000 words so far
- Read 1400000/1500000000000000000000000 words so far
- Read 1500000/1500000000000000000000000 words so far
- Read 1600000/1500000000000000000000000 words so far
- Read 1700000/1500000000000000000000000 words so far
- Read 1800000/1500000000000000000000000 words so far
- Read 1900000/1500000000000000000000000 words so far
- Read 2000000/1500000000000000000000000 words so far
- Read 2100000/1500000000000000000000000 words so far
- Read 2200000/1500000000000000000000000 words so far
- Read 2300000/1500000000000000000000000 words so far
- Read 2400000/1500000000000000000000000 words so far
- Read 2500000/1500000000000000

When choosing a reservoir size, the first relevant thing to consider is that the reservoir must be representative of the whole dataset. To do so, we ideally would choose the biggest reservoir size (literally the whole dataset), however, the whole point is to be able to save memory and resources while still being able to understand fully our data. For that reason, the best choice of reservoir size will be the the smallest reservoir size that represents the data the best possible. In other words, we will look at the biggest reservoir size, and search for the lowest size that resembles it the most.

In our case, we see that reservoir sizes 1000 and 1500 are almost identical in elements' order and have the same elements. Moreover 100 words is enough to accuratly include the top-3 words that appear in the highest size reservoir, as we have mentioned before, while 500 words will bring this accuracy up to the top-4 words. This can be observed in the results above.

The decision would really come down to how many resources we can make use of, but from my point of view 500 words would be the ideal trade-off between saving resources and accurately representing the dataset.

# 2. Determine approximately the distinct number of words

In [36]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [37]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

In [40]:
number_of_passes = 5
estimates = []

for i in range(number_of_passes):
    
    hash_function = random_hash_function()
    max_trailing_zeroes = 0

    
    for word in read_by_words(INPUT_FILE): #we read the entire file in each. We could add parameters max_words and report_every just like we did in reservoir sampling
        
        hashed_value = hash_function(word)
        trailing_zeroes = count_trailing_zeroes(hashed_value)
        max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

        
    estimate = 2 ** max_trailing_zeroes
    estimates.append(estimate)

    print("Estimate on pass %d: %d distinct words" % (i + 1, estimate))
    

Estimate on pass 1: 131072 distinct words
Estimate on pass 2: 16384 distinct words
Estimate on pass 3: 131072 distinct words
Estimate on pass 4: 32768 distinct words
Estimate on pass 5: 65536 distinct words


In [41]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 75366.4
* Median  of estimates: 65536.0


<font size="+1" color="red">Remove the limit of max words, or set to a high number, but notice that you do no need to use more than one hour of computer processing time, and perform the 10 passes. Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

In [42]:
number_of_passes = 10
estimates_10passes = []

for i in range(number_of_passes):
    
    hash_function = random_hash_function()
    max_trailing_zeroes = 0

    
    for word in read_by_words(INPUT_FILE): #limit of words was already removed.
        
        hashed_value = hash_function(word)
        trailing_zeroes = count_trailing_zeroes(hashed_value)
        max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

        
    estimate = 2 ** max_trailing_zeroes
    estimates_10passes.append(estimate)

    print("Estimate on pass %d: %d distinct words" % (i + 1, estimate))

Estimate on pass 1: 524288 distinct words
Estimate on pass 2: 32768 distinct words
Estimate on pass 3: 65536 distinct words
Estimate on pass 4: 16384 distinct words
Estimate on pass 5: 16384 distinct words
Estimate on pass 6: 2097152 distinct words
Estimate on pass 7: 131072 distinct words
Estimate on pass 8: 524288 distinct words
Estimate on pass 9: 32768 distinct words
Estimate on pass 10: 32768 distinct words


In [43]:
print("* Average of estimates: %.1f" % statistics.mean(estimates_10passes))
print("* Median  of estimates: %.1f" % statistics.median(estimates_10passes))

* Average of estimates: 347340.8
* Median  of estimates: 49152.0


After looking at the results of the 10-pass execution (and many other executions done for the sake of understanding the process) it seems obvious that the hash function randomly selected has a very high influence over the number of words estimated in every pass, with the number of trailing zeros ranging from 13 up 21 on the different observed executions. 

In our context, given the high variability of the estimates, the average will be too influenced by extreme values, such as the very large estimate in pass 6 (2097152). On the other hand, the median is less sensitive to these extreme values and provides a measure of the middle value of the distribution of estimates.

Therefore, the median is a more adequate measure of central tendency for our algorithm since it is less affected by outliers, such as 2^21 or 2^13. If we had chosen the other option, the extreme values in the estimates that significantly deviate from the typical behaviour would impact the mean disproportionately, leading to non representative estimations in some cases (when these outliers are present).

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>